In [0]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/Machine Learning/BigDataProject')

In [0]:
import numpy as np
import os
import librosa


mydata_x = []
mydata_y = []
path = '/content/drive/My Drive/Machine Learning/BigDataProject/dataset/terremoti/'
fileList = os.listdir(path)
t = 0
for i in fileList:
    t= t+1
    loaded_file = np.loadtxt(path + i)
    print(t)
    mydata_x.append(loaded_file)
    mydata_y.append([1,0,0])

In [0]:
import numpy as np
import os

path = '/content/drive/My Drive/Machine Learning/BigDataProject/dataset/scoppi/'
fileList = os.listdir(path)
t = 0
for i in fileList:
    t= t+1
    loaded_file = np.loadtxt(path + i)
    print(t)
    mydata_x.append(loaded_file)
    mydata_y.append([0,1,0])

In [0]:
import numpy as np
import os


path = '/content/drive/My Drive/Machine Learning/BigDataProject/dataset/tuoni/'
fileList = os.listdir(path)
t = 0
for i in fileList:
    t= t+1
    loaded_file = np.loadtxt(path + i)
    print(t)
    mydata_x.append(loaded_file)
    mydata_y.append([0,0,1])

In [0]:
import numpy as np
mydata_x = np.asarray(mydata_x)
mydata_y = np.asarray(mydata_y)

print(mydata_x.shape)
print(mydata_y.shape)

In [0]:
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score
import numpy as np
seed = 56

x_train, x_test, y_train, y_test = train_test_split(
    mydata_x, mydata_y, test_size=0.2, random_state=seed,stratify=mydata_y)

x_train = np.asarray(x_train)
x_test = np.asarray(x_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)


print("Training set size: ",len(x_train))
print("Test set size: ",len(x_test))
print("Training set size: ",len(y_train))
print("Test set size: ",len(y_test))

Training set size:  726
Test set size:  182
Training set size:  726
Test set size:  182


In [0]:
from keras.callbacks import ReduceLROnPlateau
from keras.utils.np_utils import to_categorical
import keras.backend as K
from keras import regularizers
from keras.layers import Lambda
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.core import Activation, Dense
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
import numpy as np
import pickle
import os
from glob import glob

num_classes = 3
AUDIO_LENGTH = 2400


# Construct model 
m = Sequential()
m.add(Conv1D(128,
              input_shape=[AUDIO_LENGTH,1],
              kernel_size=80,
              strides=4,
              padding='same',
              kernel_initializer='glorot_uniform',
              kernel_regularizer=regularizers.l2(l=0.0001)))
m.add(BatchNormalization())
m.add(Activation('relu'))
m.add(MaxPooling1D(pool_size=4, strides=None))
m.add(Conv1D(128,
              kernel_size=3,
              strides=1,
              padding='same',
              kernel_initializer='glorot_uniform',
              kernel_regularizer=regularizers.l2(l=0.0001)))
m.add(BatchNormalization())
m.add(Activation('relu'))
m.add(MaxPooling1D(pool_size=4, strides=None))
m.add(Conv1D(256,
              kernel_size=3,
              strides=1,
              padding='same',
              kernel_initializer='glorot_uniform',
              kernel_regularizer=regularizers.l2(l=0.0001)))
m.add(BatchNormalization())
m.add(Activation('relu'))
m.add(MaxPooling1D(pool_size=4, strides=None))
m.add(Conv1D(512,
              kernel_size=3,
              strides=1,
              padding='same',
              kernel_initializer='glorot_uniform',
              kernel_regularizer=regularizers.l2(l=0.0001)))
m.add(BatchNormalization())
m.add(Activation('relu'))
m.add(MaxPooling1D(pool_size=4, strides=None))
m.add(Lambda(lambda x: K.mean(x, axis=1)))  # Same as GAP for 1D Conv Layer
m.add(Dense(num_classes, activation='softmax'))

Using TensorFlow backend.


In [0]:
m.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

print(m.summary())



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 600, 128)          10368     
_________________________________________________________________
batch_normalization_1 (Batch (None, 600, 128)          512       
_________________________________________________________________
activation_1 (Activation)    (None, 600, 128)          0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 150, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 150, 128)          49280     
_________________________________________________________________
batch_normalization_2 (Batch (None, 150, 128)          512       
_________________________________________________________________
activation_2 (Activation)    (None, 150, 128)       

In [0]:
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=10, min_lr=0.0001, verbose=1)

batch_size = 128

print(x_train.shape)
x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

print(x_train.shape)

x_train=x_train.swapaxes(1,2)
x_test=x_test.swapaxes(1,2)
print(x_train.shape)

m.fit(x=x_train,
          y=y_train,
          batch_size=batch_size,
          epochs=200,
          verbose=1,
          shuffle=True,
          validation_data=(x_test, y_test),
          callbacks=[reduce_lr])

In [0]:
# Evaluating the model on the training and testing set
score = m.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = m.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  1.0
Testing Accuracy:  0.9505494485844622


Save model and weights

In [0]:
m.save('modelML/seismologicML1.model')
m.save_weights('modelML/seismologicML1.weights')